# The Iris Project
___

Feito por:

- Rafael Almada

- Rafael Rosensvaig

Projeto de Visão Computacional $\sim$
Engenharia da Computação $\sim$
Insper

In [1]:
try:
    import G6_iris_recognition
except:
    !pip install G6-iris-recognition
    import G6_iris_recognition